In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import sys
import warnings
from time import time

In [82]:
cd

/Users/karatechop


In [83]:
cd /Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data

/Users/karatechop/Library/CloudStorage/GoogleDrive-kc1558628@berkeley.edu/My Drive/ENERES 131 FINAL PROJECT (1)/data


In [6]:
ls

2008_2022_HAB_nutrients_scripps - data.csv
2008_2022_HAB_nutrients_scripps.gsheet
buoy/
original 100 year scripps temp data/
recent scripps_TEMP_2005-2020 from 100 year.csv


In [182]:
nutrients = pd.read_html('https://erddap.sccoos.org/erddap/tabledap/HABs-ScrippsPier.htmlTable?Location_Code%2Clatitude%2Clongitude%2Cdepth%2CSampleID%2Ctime%2CTemp%2CAir_Temp%2CSalinity%2CChl_Volume_Filtered%2CChl1%2CChl2%2CAvg_Chloro%2CPhaeo1%2CPhaeo2%2CAvg_Phaeo%2CPhosphate%2CSilicate%2CNitrite%2CNitrite_Nitrate%2CAmmonium%2CNitrate%2CDA_Volume_Filtered%2CpDA%2CtDA%2CdDA%2CVolume_Settled_for_Counting%2CAkashiwo_sanguinea%2CAlexandrium_spp%2CDinophysis_spp%2CLingulodinium_polyedra%2CProrocentrum_spp%2CPseudo_nitzschia_delicatissima_group%2CPseudo_nitzschia_seriata_group%2CCeratium_spp%2CCochlodinium_spp%2CGymnodinium_spp%2COther_Diatoms%2COther_Dinoflagellates%2CTotal_Phytoplankton&time%3E=2008-01-24T00%3A00%3A00Z&time%3C=2022-10-31T17%3A43%3A00Z')[1]
nutrients = nutrients[nutrients.columns[1:]]

In [183]:
nutrients['hab_RT_status','categorical 0/1'] = 0
nutrients.loc[(nutrients['Chl1',              'mg/m3']>nutrients['Chl1',              'mg/m3'].quantile(0.95)) | 
              (nutrients['Chl2',              'mg/m3']>nutrients['Chl2',              'mg/m3'].quantile(0.95)) | 
              (nutrients['Avg_Chloro',              'mg/m3']>nutrients['Avg_Chloro',              'mg/m3'].quantile(0.95)),('hab_RT_status', 'categorical 0/1')]=1

nutrients['hab_RT_status','categorical 0/1'].value_counts()

0    709
1     40
Name: (hab_RT_status, categorical 0/1), dtype: int64

In [87]:
nutrients.isna().sum()

latitude                              degrees_north           0
longitude                             degrees_east            0
depth                                 m                     749
SampleID                              Unnamed: 4_level_1      0
time                                  UTC                     0
Temp                                  degree_C                0
Air_Temp                              degree_C              749
Salinity                              Unnamed: 8_level_1    749
Chl_Volume_Filtered                   mL                      9
Chl1                                  mg/m3                  10
Chl2                                  mg/m3                  11
Avg_Chloro                            mg/m3                  10
Phaeo1                                mg/m3                  10
Phaeo2                                mg/m3                  11
Avg_Phaeo                             mg/m3                  10
Phosphate                             uM

In [99]:
nutrients_simplified = nutrients[['time','Temp',
                                  'Chl1','Chl2','Avg_Chloro',
                                  'Phaeo1','Phaeo2','Avg_Phaeo',
                                  'Phosphate',
                                  'Silicate',
                                  'Nitrite',
                                  'Ammonium',
                                  'Total_Phytoplankton',
                                  'hab_RT_status']]
nutrients_simplified

,time,Temp,Chl1,Chl2,Avg_Chloro,Phaeo1,Phaeo2,Avg_Phaeo,Phosphate,Silicate,Nitrite,Ammonium,Total_Phytoplankton,hab_RT_status
,UTC,degree_C,mg/m3,mg/m3,mg/m3,mg/m3,mg/m3,mg/m3,uM,uM,uM,uM,cells/L,categorical 0/1
0,2008-06-30T15:00:00Z,19.8,1.81,1.83,1.82,0.59,0.61,0.60,0.24,5.6,0.03,0.53,NaN,0
1,2008-07-07T15:00:00Z,17.3,2.68,2.73,2.71,1.24,1.12,1.18,0.37,5.1,0.04,1.06,NaN,0
2,2008-07-14T14:54:00Z,21.7,0.99,1.00,0.99,0.55,0.54,0.55,0.24,3.5,0.00,0.35,NaN,0
3,2008-07-21T14:50:00Z,20.2,1.21,1.24,1.23,0.49,0.48,0.48,0.17,4.2,0.00,0.55,NaN,0
4,2008-07-28T16:44:00Z,22.4,2.37,2.44,2.41,0.41,0.36,0.39,0.12,3.8,0.00,0.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,2022-10-03T19:02:00Z,19.8,6.83,6.86,6.84,0.58,0.56,0.57,NaN,NaN,NaN,NaN,239056.0,1
745,2022-10-10T18:53:00Z,20.3,10.80,10.80,10.80,1.11,1.14,1.12,NaN,NaN,NaN,NaN,456209.0,1
746,2022-10-17T19:01:00Z,20.2,14.00,14.06,14.03,1.83,2.69,2.26,NaN,NaN,NaN,NaN,657092.0,1


In [179]:
nutrients_simplified.to_csv('nutrients_simplified(UNITS).csv',index=False)
# nutrients_simplified.droplevel(1,axis=1).to_csv('nutrients_simplified(NO_units).csv',index=False)